In [1]:
def cartesian_product(x,y):
    '''
    This function will take two iterables and returns 
    the cartesian product of them as a list
    '''
    return [a+b for a in x for b in y]

In [2]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    print('')
    
    rows = 'ABCDEFGHI'
    cols = '123456789'
    boxes = cartesian_product(rows, cols)
    
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                    for c in cols))
        if r in 'CF': print(line)
    return

In [3]:
def eliminate(grid_dict):
    for k,v in grid_dict.items():
        if len(v) != 1: # if the box needs elimination
            peers = peer_dict[k]    # get all the peers
            peer_values = set([grid_dict[p] for p in peers if len(grid_dict[p]) == 1])
            grid_dict[k] = ''.join(set(grid_dict[k]) - peer_values)
    return grid_dict

In [4]:
def only_choice(grid_dict):
    for unit in unit_list:
        for num in '123456789':
            num_places = [box for box in unit if num in grid_dict[box]]
            if len(num_places) == 1:
                grid_dict[num_places[0]] = num
    return grid_dict

In [5]:
def naked_twins(grid_dict):
    for unit in unit_list:
        
        # slice the grid_dict to contain only the boxes in the unit
        values = dict([[box, ''.join(sorted(grid_dict[box]))] for box in unit])
        
        # find all the items with 2-digit values
        double_digits = dict([[box, values[box]] for box in values if len(values[box])==2])
        
        # check if any of those 2-digit values occur exactly twice
        double_digits_occuring_twice = dict([[box, val] for box, val in double_digits.items() if list(double_digits.values()).count(val)==2])
        
        if len(double_digits_occuring_twice.items()) != 0:
            # reverse the dictionary to get the key-pairs easily
            reverse_dict = {}
            for k, v in double_digits_occuring_twice.items():
                reverse_dict.setdefault(v, []).append(k)

            # it is a list of 2 items(keys | boxes) only
            naked_twins = list(reverse_dict.items())[0][1]

            # remove the naked_twins digits from other boxes in the unit
            for k,v in values.items():
                if (k not in naked_twins) and (len(v) > 1):
                    values[k] = ''.join(set(values[k]) - set(values[naked_twins[0]]))
    
        # replace the values in grid_dict with the updated values
        for k,v in values.items():
            grid_dict[k] = v
    
    return grid_dict

#         double_digits = dict([[box, values.values().count(box)] for box in values.values() if len(box)==2])
#         naked_twins = [k for k,count in double_digits.items() if count==2]
        

In [6]:
def run_episode(grid_dict):
    stuck = False
    while not stuck:
        # Check how many boxes have a fixed value
        solved_values_before = len([box for box in grid_dict.keys() if len(grid_dict[box]) == 1])
        
        # Use the Eliminate Strategy
        grid_dict = eliminate(grid_dict)
        
        # Use the Only Choice Strategy
        grid_dict = only_choice(grid_dict)
        
        # Use the Naked Twins Strategy
        grid_dict = naked_twins(grid_dict)
        
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in grid_dict.keys() if len(grid_dict[box]) == 1])
        
        # If no new values were added, stop the loop.
        stuck = solved_values_before == solved_values_after
        
        
        # if the current sudoku configuration is un-solvable then return False
        if len([box for box in grid_dict.keys() if len(grid_dict[box]) == 0]):
            return False 
    return grid_dict

In [7]:
def search(grid_dict):
    grid_dict = run_episode(grid_dict)
    
    if grid_dict is False:
        return False
    
    if all(len(v) == 1 for k,v in grid_dict.items()): 
        return grid_dict
    
    # Choose one of the unfilled squares with the fewest possibilities
    length,k = min((len(val), key) for key,val in grid_dict.items() if len(val) > 1)
    # print(k, length)
    
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for digit in grid_dict[k]:
        new_sudoku = dict(list(grid_dict.items()))
        new_sudoku[k] = digit
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [8]:
if __name__ == '__main__':
    
    # this is the worlds' hardest sudoku puzzle
    start = '8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..'
    # start = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
    
    rows = 'ABCDEFGHI'
    cols = '123456789'
    boxes = cartesian_product(rows, cols)

    row_units = [cartesian_product(r, cols) for r in rows]
    col_units = [cartesian_product(rows, c) for c in cols]
    box_units = [cartesian_product(r,c) 
                    for r in ['ABC', 'DEF', 'GHI'] 
                    for c in ['123','456','789']]

    unit_list = row_units + col_units + box_units

    # each box(key) with its units(value)
    unit_dict = dict((box, [unit for unit in unit_list if box in unit]) for box in boxes)
    
    # each box with its peers
    peer_dict = dict((box, set(sum(unit_dict[box], [])) - set([box])) for box in boxes)

    # start string converted to dictionary
    assert len(start) == 81
    grid_dict = dict(zip(boxes, start))

    # replacing the dots(.) with '123456789' (possible values in the box)
    for k,v in grid_dict.items():
        if v == '.':
            grid_dict[k] = '123456789'
            
    solved_grid = search(grid_dict)
    
    display(solved_grid)


8 1 4 |2 5 3 |6 9 7 
9 2 3 |6 7 8 |1 5 4 
5 7 6 |4 9 1 |2 8 3 
------+------+------
3 5 2 |9 6 7 |8 4 1 
1 8 9 |3 4 5 |7 2 6 
4 6 7 |1 8 2 |9 3 5 
------+------+------
2 4 1 |7 3 9 |5 6 8 
7 6 8 |5 2 4 |3 1 9 
3 9 5 |8 1 6 |4 7 2 


In [9]:
# display(grid_dict)

In [10]:
# new_grid = eliminate(grid_dict)

In [11]:
# display(new_grid)

In [12]:
# naked_dict = naked_twins(new_grid)

In [13]:
# display(naked_dict)